利用vgg提取每一类的平均的特征，保存特征

In [12]:
import os
import glob
import json
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
#from progressbar import * #进度条
from tqdm import tqdm #进度条
from torchnet import meter
from natsort import natsorted
import matplotlib.pyplot as plt
import torch.nn.functional as F #torch是关于运算的包
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集
os.environ['CUDA_VISIBLE_DEVICES']='2'

### Vgg模型 

In [3]:
#vgg_model = models.vgg16(pretrained=True)
#print(vgg_model)
class Encoder(nn.Module):
    '''
    提取vgg的4096向量
    '''
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg16(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output

In [19]:
#print(Encoder())
def extractor(img_path, net):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    img = Image.open(img_path).convert("RGB")
    img = transform(img)
    img = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)#增加一维
    
    net = net.cuda()
    img = img.cuda()
    y = net(img).cpu()
    y = torch.squeeze(y)
    y = y.data.numpy()
    
    return y

In [21]:
img_path = 'data/train/1cf255a78.jpg'
feature = extractor(img_path, Encoder())
print(feature.shape)

(4096,)


### 提取特征

In [22]:
def extract_train_feature(dir_path, model, feature_dim=4096):
    data = np.zeros((1, feature_dim))
    label = []
    for file in tqdm(os.listdir(dir_path)):
        if file == 'new_whale':
            continue
        path = os.path.join(dir_path, file)
        if os.path.isdir(path):
            feature_tmp = np.zeros((1, feature_dim))
            label_name = file
            img_num = 0
            for img in os.listdir(path):
                img_path = os.path.join(path, img)
                feature = extractor(img_path, model)
                img_num += 1
                feature_tmp += feature
            if img_num > 0:
                feature = feature_tmp / img_num
                data = np.concatenate((data, feature))
                label.append(label_name)
    data = data[1:, :]
    np.savetxt('model/train_feature.txt', data, fmt='%f')
    label_file = open('model/label.txt', 'w')
    json.dump(label, label_file)
    label_file.close

In [23]:
extract_train_feature('data/train_classes', Encoder())

100%|██████████| 5005/5005 [10:12<00:00,  8.17it/s]


### 找到最近的5个类别

In [ ]:
def top_5_classes(test_feature):
    train_features = np.loadtxt('model/train_feature.txt', dtype=float)
    train_labels = open('model/label.txt', 'r')
    labels = json.load(train_labels)
    train_labels.close()
    
    distance = train_features - test